# Initialization
Check if Colaboratory is empty, if yes, go to next step, if not, kill vm instance and reopen notebook

In [0]:
!ls -l

In [0]:
!kill -9 -1

# Install required libraries

In [0]:
!apt-get install -y -qq protobuf-compiler python-pil python-lxml libsm6 libxext6 && pip install -q -U opencv-python

In [0]:
!git clone --quiet https://github.com/tensorflow/models.git tensorflow_models

In [0]:
cd tensorflow_models/research

In [0]:
!protoc object_detection/protos/*.proto --python_out=.

In [0]:
import sys
sys.path.append('/content/tensorflow_models/research')
sys.path.append('/content/tensorflow_models/research/slim')

In [0]:
cd

# Upload from Google Drive (substitute 3 next uploading steps)
Generic zip file upload from Google Drive, in case of needs. Set Google Drive (fileId, unzipDir) into fileIdUnzipDirList to upload and unzip it.

In [0]:
!pip install -U -q PyDrive

fileIdUnzipDirList = [('19PSIC2rtps2r5DHwqTEUtRlccpaKZW4V', 'images'), ('1SuSic7drlmOaHvLoEj9QXY-ai5e3ftK7', 'model'), ('12MqnYTyueAu4cJx9IpN3aZy6qbqLnJ1_','annotations')]

import os
from zipfile import ZipFile
from shutil import rmtree
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fileName = 'downloaded.zip'
for fileId, unzipDir in fileIdUnzipDirList:
    downloaded = drive.CreateFile({'id': fileId})
    downloaded.GetContentFile(fileName)
    print('Downloaded file id: ' + fileId + ' with file name: ' + fileName)

    if os.path.isdir(unzipDir):
        rmtree(unzipDir)
    os.mkdir(unzipDir)
    ds = ZipFile(fileName)
    ds.extractall(unzipDir)
    os.remove(fileName)
    print('Extracted zip file ' + fileName + ' in directory: ' + unzipDir)


# Import training image dataset
In the next cell upload a zip file, containing files in .jpg format

In [0]:
from google.colab import files
from zipfile import ZipFile
from shutil import rmtree
import os

datadir = 'images'
if os.path.isdir(datadir):
  rmtree(datadir)
os.mkdir(datadir)

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
  
for name, data in uploaded.items():
  with open(name, 'wb') as f:
    f.write(data)
    f.close()
    print('saved file ' + name)
    ds = ZipFile(name)
    ds.extractall(datadir)
    print('extracted file ' + name)

# Import training annotation files, created with LabelImg
In the next cell upload a zip file, containing files in .xml format

In [0]:
from google.colab import files
from zipfile import ZipFile
from shutil import rmtree
import os

datadir = 'annotations'
if os.path.isdir(datadir):
  rmtree(datadir)
os.mkdir(datadir)

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
  
for name, data in uploaded.items():
  with open(name, 'wb') as f:
    f.write(data)
    f.close()
    print('saved file ' + name)
    ds = ZipFile(name)
    ds.extractall(datadir)
    print('extracted file ' + name)

# Import model and config file from ModelZoo
In the next cell upload a zip file, containing 4 files:
* 3 files from pretrained models in ModelZoo: model.ckpt.data-00000-of-00001 model.ckpt.index model.ckpt.meta
* 1 config file pets version (i.e. faster_rcnn_inception_v2_pets.config) from https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs

In [0]:
from google.colab import files
from zipfile import ZipFile
from shutil import rmtree
import os

datadir = 'model'
if os.path.isdir(datadir):
  rmtree(datadir)
os.mkdir(datadir)

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
  
for name, data in uploaded.items():
  with open(name, 'wb') as f:
    f.write(data)
    f.close()
    print('saved file ' + name)
    ds = ZipFile(name)
    ds.extractall(datadir)
    print('extracted file ' + name)

# Create labels.csv from xml annotation files

In [0]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    datadir = 'data'
    if os.path.isdir(datadir):
        rmtree(datadir)
    os.mkdir(datadir)
    image_path = 'annotations'
    xml_df = xml_to_csv(image_path)
    xml_df.to_csv('data/labels.csv', index=None)
    print('Successfully converted xml to csv.')


main()

# Split labels between train and test data
Every 10 lines we split 80% of labels for training and 20% for test

In [0]:
with open('data/labels.csv', 'r') as f_in:
  f_train =  open('data/train_labels.csv', 'w')
  f_test =  open('data/test_labels.csv', 'w')

  first_line = f_in.readline()
  f_train.write(first_line)
  f_test.write(first_line)
    
  for line_number, line in enumerate(f_in):
    if line_number % 10 < 8:
      f_train.write(line)
    else:
      f_test.write(line)

  f_train.close()
  f_test.close()
  
print('Successfully created train and test labels files.')

# Create labels class files: labels.csv and label_map.pbtxt

In [0]:
import pandas as pd
import json

labels = pd.read_csv('data/labels.csv', delimiter=',')
classes = labels['class'].unique()

with open('data/classes.csv', 'w') as f:
    for s in classes:
        f.write(str(s) +',')

with open('data/label_map.pbtxt', 'w') as f:
    index_classes = ''
    for line_number, line in enumerate(classes):
        index_classes += "item {\n  id: " + str(line_number + 1) + "\n  name: '" + line + "'\n}\n"
    f.write(index_classes)

print('Successfully created classes files.')

# Create train and test data files

In [0]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import csv
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

flags = tf.app.flags
# flags.DEFINE_string('csv_input', '', 'Path to the CSV input')
# flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
FLAGS = flags.FLAGS

label_files = ['data/train_labels.csv', 'data/test_labels.csv']
output_files = ['data/train.record', 'data/test.record']


def class_text_to_int(row_label):
    with open('data/classes.csv', 'r') as classes_file:
        classes_list = classes_file.readline().split(',')
    index = classes_list.index(row_label) + 1
    return index

    
def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main():
  for i in range(len(label_files)):
    writer = tf.python_io.TFRecordWriter(output_files[i])
    path = os.path.join(os.getcwd(), 'images')
    examples = pd.read_csv(label_files[i])
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), output_files[i])
    print('Successfully created the TFRecords: {}'.format(output_path))


#if __name__ == '__main__':
#    tf.app.run()
main()

# Edit config file to set num_classes
It must to be = classes number

In [0]:
import re

filename = 'model/faster_rcnn_inception_v2_pets.config'

with open('data/classes.csv', 'r') as classes_file:
    classes_list = classes_file.readline().split(',')
classes_number = len(classes_list) - 1

with open(filename, 'r') as f:
    s = f.read()
with open(filename, 'w') as f:
    s = re.sub('num_classes: [0-9]+', 'num_classes: ' + str(classes_number), s)
    f.write(s)
    
print('Finished set num_classes: ' + str(classes_number))

# Further settings of config file
If you use other config file then proposed, edit file to set PATH_TO_BE_CONFIGURED:
* fine_tune_checkpoint: "./model/model.ckpt"
* input_path: "./data/train.record"
* label_map_path: "./data/label_map.pbtxt"	
* input_path: "./data/test.record"
* label_map_path: "./data/label_map.pbtxt"


# Train model

In [0]:
# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

r"""Training executable for detection models.

This executable is used to train DetectionModels. There are two ways of
configuring the training job:

1) A single pipeline_pb2.TrainEvalPipelineConfig configuration file
can be specified by --pipeline_config_path.

Example usage:
    ./train \
        --logtostderr \
        --train_dir=path/to/train_dir \
        --pipeline_config_path=pipeline_config.pbtxt

2) Three configuration files can be provided: a model_pb2.DetectionModel
configuration file to define what type of DetectionModel is being trained, an
input_reader_pb2.InputReader file to specify what training data will be used and
a train_pb2.TrainConfig file to configure training parameters.

Example usage:
    ./train \
        --logtostderr \
        --train_dir=path/to/train_dir \
        --model_config_path=model_config.pbtxt \
        --train_config_path=train_config.pbtxt \
        --input_config_path=train_input_config.pbtxt
"""

import functools
import json
import os
import tensorflow as tf

from object_detection import trainer
from object_detection.builders import dataset_builder
from object_detection.builders import model_builder
from object_detection.utils import config_util
from object_detection.utils import dataset_util

tf.logging.set_verbosity(tf.logging.INFO)

flags = tf.app.flags
#flags.DEFINE_string('master', '', 'Name of the TensorFlow master to use.')
FLAGS_master = ''
#flags.DEFINE_integer('task', 0, 'task id')
FLAGS_task = 0
#flags.DEFINE_integer('num_clones', 1, 'Number of clones to deploy per worker.')
FLAGS_num_clones = 1
#flags.DEFINE_boolean('clone_on_cpu', False,
#                     'Force clones to be deployed on CPU.  Note that even if '
#                     'set to False (allowing ops to run on gpu), some ops may '
#                     'still be run on the CPU if they have no GPU kernel.')
FLAGS_clone_on_cpu = False
#flags.DEFINE_integer('worker_replicas', 1, 'Number of worker+trainer '
#                     'replicas.')
FLAGS_worker_replicas = 1
#flags.DEFINE_integer('ps_tasks', 0,
#                     'Number of parameter server tasks. If None, does not use '
#                     'a parameter server.')
FLAGS_ps_tasks = 0
#flags.DEFINE_string('train_dir', '',
#                    'Directory to save the checkpoints and training summaries.')
FLAGS_train_dir = 'data/train'
#flags.DEFINE_string('pipeline_config_path', '',
#                    'Path to a pipeline_pb2.TrainEvalPipelineConfig config '
#                    'file. If provided, other configs are ignored')
FLAGS_pipeline_config_path = 'model/faster_rcnn_inception_v2_pets.config'
#flags.DEFINE_string('train_config_path', '',
#                    'Path to a train_pb2.TrainConfig config file.')
FLAGS_train_config_path = ''
#flags.DEFINE_string('input_config_path', '',
#                    'Path to an input_reader_pb2.InputReader config file.')
FLAGS_input_config_path = ''
#flags.DEFINE_string('model_config_path', '',
#                    'Path to a model_pb2.DetectionModel config file.')
FLAGS_model_config_path = ''
FLAGS = flags.FLAGS


def main():
  assert FLAGS_train_dir, '`train_dir` is missing.'
  if FLAGS_task == 0: tf.gfile.MakeDirs(FLAGS_train_dir)
  if FLAGS_pipeline_config_path:
    configs = config_util.get_configs_from_pipeline_file(
        FLAGS_pipeline_config_path)
    if FLAGS_task == 0:
      tf.gfile.Copy(FLAGS_pipeline_config_path,
                    os.path.join(FLAGS_train_dir, 'pipeline.config'),
                    overwrite=True)
  else:
    configs = config_util.get_configs_from_multiple_files(
        model_config_path=FLAGS_model_config_path,
        train_config_path=FLAGS_train_config_path,
        train_input_config_path=FLAGS_input_config_path)
    if FLAGS_task == 0:
      for name, config in [('model.config', FLAGS_model_config_path),
                           ('train.config', FLAGS_train_config_path),
                           ('input.config', FLAGS_input_config_path)]:
        tf.gfile.Copy(config, os.path.join(FLAGS_train_dir, name),
                      overwrite=True)

  model_config = configs['model']
  train_config = configs['train_config']
  input_config = configs['train_input_config']

  model_fn = functools.partial(
      model_builder.build,
      model_config=model_config,
      is_training=True)

  def get_next(config):
    return dataset_util.make_initializable_iterator(
      dataset_builder.build(config)).get_next()

  create_input_dict_fn = functools.partial(get_next, input_config)

  env = json.loads(os.environ.get('TF_CONFIG', '{}'))
  cluster_data = env.get('cluster', None)
  cluster = tf.train.ClusterSpec(cluster_data) if cluster_data else None
  task_data = env.get('task', None) or {'type': 'master', 'index': 0}
  task_info = type('TaskSpec', (object,), task_data)

  # Parameters for a single worker.
  ps_tasks = 0
  worker_replicas = 1
  worker_job_name = 'lonely_worker'
  task = 0
  is_chief = True
  master = ''

  if cluster_data and 'worker' in cluster_data:
    # Number of total worker replicas include "worker"s and the "master".
    worker_replicas = len(cluster_data['worker']) + 1
  if cluster_data and 'ps' in cluster_data:
    ps_tasks = len(cluster_data['ps'])

  if worker_replicas > 1 and ps_tasks < 1:
    raise ValueError('At least 1 ps task is needed for distributed training.')

  if worker_replicas >= 1 and ps_tasks > 0:
    # Set up distributed training.
    server = tf.train.Server(tf.train.ClusterSpec(cluster), protocol='grpc',
                             job_name=task_info.type,
                             task_index=task_info.index)
    if task_info.type == 'ps':
      server.join()
      return

    worker_job_name = '%s/task:%d' % (task_info.type, task_info.index)
    task = task_info.index
    is_chief = (task_info.type == 'master')
    master = server.target

  trainer.train(create_input_dict_fn, model_fn, train_config, master, task,
                FLAGS_num_clones, worker_replicas, FLAGS_clone_on_cpu, ps_tasks,
                worker_job_name, is_chief, FLAGS_train_dir)


#if __name__ == '__main__':
#  tf.app.run()
main()

# Export trained model

In [0]:
# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

r"""Tool to export an object detection model for inference.

Prepares an object detection tensorflow graph for inference using model
configuration and an optional trained checkpoint. Outputs inference
graph, associated checkpoint files, a frozen inference graph and a
SavedModel (https://tensorflow.github.io/serving/serving_basic.html).

The inference graph contains one of three input nodes depending on the user
specified option.
  * `image_tensor`: Accepts a uint8 4-D tensor of shape [None, None, None, 3]
  * `encoded_image_string_tensor`: Accepts a 1-D string tensor of shape [None]
    containing encoded PNG or JPEG images. Image resolutions are expected to be
    the same if more than 1 image is provided.
  * `tf_example`: Accepts a 1-D string tensor of shape [None] containing
    serialized TFExample protos. Image resolutions are expected to be the same
    if more than 1 image is provided.

and the following output nodes returned by the model.postprocess(..):
  * `num_detections`: Outputs float32 tensors of the form [batch]
      that specifies the number of valid boxes per image in the batch.
  * `detection_boxes`: Outputs float32 tensors of the form
      [batch, num_boxes, 4] containing detected boxes.
  * `detection_scores`: Outputs float32 tensors of the form
      [batch, num_boxes] containing class scores for the detections.
  * `detection_classes`: Outputs float32 tensors of the form
      [batch, num_boxes] containing classes for the detections.
  * `detection_masks`: Outputs float32 tensors of the form
      [batch, num_boxes, mask_height, mask_width] containing predicted instance
      masks for each box if its present in the dictionary of postprocessed
      tensors returned by the model.

Notes:
 * This tool uses `use_moving_averages` from eval_config to decide which
   weights to freeze.

Example Usage:
--------------
python export_inference_graph \
    --input_type image_tensor \
    --pipeline_config_path path/to/ssd_inception_v2.config \
    --trained_checkpoint_prefix path/to/model.ckpt \
    --output_directory path/to/exported_model_directory

The expected output would be in the directory
path/to/exported_model_directory (which is created if it does not exist)
with contents:
 - graph.pbtxt
 - model.ckpt.data-00000-of-00001
 - model.ckpt.info
 - model.ckpt.meta
 - frozen_inference_graph.pb
 + saved_model (a directory)
"""
import tensorflow as tf
from google.protobuf import text_format
from object_detection import exporter
from object_detection.protos import pipeline_pb2

slim = tf.contrib.slim
flags = tf.app.flags

import glob
newest = max(glob.iglob('data/train/model.ckpt*.meta'), key=os.path.getctime)
model_prefix = newest[:newest.rfind('.')]
print ('Model_prefix: ' + model_prefix)

#flags.DEFINE_string('input_type', 'image_tensor', 'Type of input node. Can be '
#                    'one of [`image_tensor`, `encoded_image_string_tensor`, '
#                    '`tf_example`]')
FLAGS_input_type = 'image_tensor'
#flags.DEFINE_string('input_shape', None,
#                    'If input_type is `image_tensor`, this can explicitly set '
#                    'the shape of this input tensor to a fixed size. The '
#                    'dimensions are to be provided as a comma-separated list '
#                    'of integers. A value of -1 can be used for unknown '
#                    'dimensions. If not specified, for an `image_tensor, the '
#                    'default shape will be partially specified as '
#                    '`[None, None, None, 3]`.')
FLAGS_input_shape = None
#flags.DEFINE_string('pipeline_config_path', None,
#                    'Path to a pipeline_pb2.TrainEvalPipelineConfig config '
#                    'file.')
FLAGS_pipeline_config_path = 'model/faster_rcnn_inception_v2_pets.config'
#flags.DEFINE_string('trained_checkpoint_prefix', None,
#                    'Path to trained checkpoint, typically of the form '
#                    'path/to/model.ckpt')
FLAGS_trained_checkpoint_prefix  = model_prefix
#flags.DEFINE_string('output_directory', None, 'Path to write outputs.')
FLAGS_output_directory = 'data/fine_tuned_model'

#tf.app.flags.mark_flag_as_required('pipeline_config_path')
#tf.app.flags.mark_flag_as_required('trained_checkpoint_prefix')
#tf.app.flags.mark_flag_as_required('output_directory')
FLAGS = flags.FLAGS


def main():
  pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
  with tf.gfile.GFile(FLAGS_pipeline_config_path, 'r') as f:
    text_format.Merge(f.read(), pipeline_config)
  if FLAGS_input_shape:
    input_shape = [
        int(dim) if dim != '-1' else None
        for dim in FLAGS_input_shape.split(',')
    ]
  else:
    input_shape = None
  exporter.export_inference_graph(FLAGS_input_type, pipeline_config,
                                  FLAGS_trained_checkpoint_prefix,
                                  FLAGS_output_directory, input_shape)
  print ('Model export complete in: ' + FLAGS_output_directory)
  

#if __name__ == '__main__':
#  tf.app.run()
main()

#Upload jpg image to execute object detection

In [0]:
from google.colab import files
from os import path

uploaded = files.upload()
  
for name, data in uploaded.items():
  with open('img.jpg', 'wb') as f:
    f.write(data)
    f.close()
    print('saved file ' + name)

# Execute object detection

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import argparse
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util


PATH_TO_CKPT = 'data/fine_tuned_model/frozen_inference_graph.pb'
PATH_TO_LABELS = 'data/label_map.pbtxt'

TEST_IMAGE_PATHS = 'img.jpg'
IMAGE_SIZE = (12, 8)


with open(PATH_TO_LABELS, 'r') as classes_file:
    content = classes_file.read()
num_classes = content.count('item {')
print('Set num_classes: ' + str(num_classes))

a = argparse.ArgumentParser()
a.add_argument("--image", default="img.jpg", help="path to image")
args = a.parse_args([])
image_np = cv2.imread(args.image)

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)


with detection_graph.as_default():
  #DISABLE GPU IF OOM ERROR
#  config = tf.ConfigProto(device_count = {'GPU': 0})
#  with tf.Session(config=config, graph=detection_graph) as sess:
  with tf.Session(graph=detection_graph) as sess:
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    image_np_expanded = np.expand_dims(image_np, axis=0)
    (boxes, scores, classes, num) = sess.run(
        [detection_boxes, detection_scores, detection_classes, num_detections],
        feed_dict={image_tensor: image_np_expanded})
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8)

    img_result = 'result.jpg'
    cv2.imwrite(img_result, cv2.resize(image_np, (800,600)))
    display(Image.open(img_result))

# Export trained model

In [0]:
from google.colab import files

files.download('data/label_map.pbtxt')
files.download('data/fine_tuned_model/frozen_inference_graph.pb')

print('Downloaded model files')